<a href="https://colab.research.google.com/github/super-creek/project/blob/main/Piano_transcription_inference_JP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ##**①必要なソフトのインストール** { display-mode: "form" }
!pip install yt_dlp
!pip install ffmpeg
!pip install librosa==0.9.2

import yt_dlp
import ffmpeg
import os
import os.path as osp
from os import path

!pip install piano_transcription_inference
from IPython.display import Audio
from google.colab import files

!rm -r sample_data

In [ ]:
#@title ##**②音声ファイルを読み込む** { display-mode: "form" }
#@markdown mid化したい音声のURLを貼ろう！(YouTube,Twitterなど)
source_url = '' #@param {type:"string"}
#@markdown 空欄のまま実行するとPCからファイルを読み込めます※mp3ファイルでアップロードしてください

if source_url == '':
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  os.rename(fn, fn.replace(" ", ""))
  fn = fn.replace(" ", "")
  file_name = "music." + fn.split(".")[-1]
  !mv -f $fn $file_name

else:
  try:
    ydl_opts = {
        'format': 'bestaudio[ext=m4a]/mp3',
        'outtmpl': 'music.mp3',
        }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
      ydl.download([source_url])
    file_name = 'music.mp3'

  except BaseException:
    !wget $source_url
    fn = source_url.split('/')[-1]
    os.rename(fn, fn.replace(" ", ""))
    fn = fn.replace(" ", "")
    file_name = "music." + fn.split(".")[-1]
    !mv -f $fn $file_name

In [ ]:
#@title ##**③出力するファイルの名前を書いてね** { display-mode: "form" }
#@markdown ※拡張子を書く必要はありません
output_midi_path = '' #@param {type:"string"}
from piano_transcription_inference import PianoTranscription, sample_rate, load_audio

def transcribe(audio_path, output_midi_path):
    # Load audio
    audio, _ = load_audio(audio_path, sr=sample_rate, mono=True)

    # Transcriptor
    transcriptor = PianoTranscription(device='cuda', checkpoint_path=None)

    # Transcribe and write out to MIDI file
    transcriptor.transcribe(audio, output_midi_path)

In [ ]:
#@title ##**④書き出し開始！** { display-mode: "form" }
Audio('music.mp3')
transcribe("music.mp3",output_midi_path + ".mid")
files.download(output_midi_path + ".mid")
!rm music.mp3